In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.utils.class_weight import compute_sample_weight
import seaborn as sns
import neurokit2 as nk
from tqdm import tqdm
from scipy import mean, count_nonzero
from biosppy.signals import ecg
import warnings
import pprint
import biosppy
import pywt
warnings.filterwarnings("ignore")
np.random.seed(1234)
rnd = 1234

## Read Data

In [ ]:
X = pd.read_csv('X_train.csv')
test = pd.read_csv('X_test.csv')
y = pd.read_csv('y_train.csv')

test_ids = test['id']
X.drop(columns='id', inplace=True)
y.drop(columns='id', inplace=True)
test.drop(columns='id', inplace=True)

## Clean raw ecg

In [ ]:
X_clean = [item[~np.isnan(item)] for item in X.values]

def clean_raw_ecg(ecg_raw):
    return nk.ecg_clean(ecg_raw, sampling_rate=300, method="neurokit")

X_ecg = list(map(clean_raw_ecg, X_clean))

## Create features

### R peaks and heartbeat stats

In [ ]:
r_peaks = []
mean_heart = []
std_heart = []
median_heart = []
for idx, signal in tqdm(enumerate(X_ecg)):
    _, _, peak, _, templates, _, _ = ecg.ecg(signal=signal, sampling_rate=300, show=False)
    r_peaks.append(peak)
    mean_heart.append(np.mean(templates, axis=0))
    std_heart.append(np.std(templates, axis=0))
    median_heart.append(np.median(templates, axis=0))

In [ ]:
# collect into one numpy array the above lists
mean_heart_full = np.vstack(mean_heart)
std_heart_full = np.vstack(std_heart)
median_heart_full = np.vstack(median_heart)

In [ ]:
# extract values of the respective locations of the R-peaks identified
peak_values = []
for idx, val in enumerate(X_ecg):
    peaks = np.array(r_peaks[idx])
    peak_values.append((np.array(X_ecg[idx][peaks])))        

In [ ]:
# extract stats about the r-peaks
rpeakmean = np.array([item.mean() for item in peak_values])
rpeakmedian = np.array([np.median(item) for item in peak_values])
rpeakmstd = np.array([item.std() for item in peak_values])
rpeakmax = np.array([item.max() for item in peak_values])
rpeakmin = np.array([item.min() for item in peak_values])

In [ ]:
# extract features related to heartbeat
mean_h = np.median(mean_heart_full, axis=1)
std_h = np.median(std_heart_full, axis=1)
median_h = np.median(median_heart_full, axis=1)
max_mean_h = np.max(mean_heart_full, axis = 1)
min_mean_h = np.min(mean_heart_full, axis = 1)
max_std_h = np.max(std_heart_full, axis = 1)
min_std_h = np.min(std_heart_full, axis = 1)

### Heart rate variabilty time domain stats

In [ ]:
hrv_time_domain = pd.DataFrame(nk.hrv_time(r_peaks[0], sampling_rate=300))
idx_list = []
for idx, peak in enumerate(r_peaks[1:]):
    hrv_time_domain = pd.concat([hrv_time_domain, nk.hrv_time(peak, sampling_rate=300)])

hrv_time_domain = hrv_time_domain.dropna(axis=1)
hrv_time_domain = np.array(hrv_time_domain)

In [ ]:
## heartrate
heart_rate_mean = []
heart_rate_std = []
heart_rate_median = []
for idx, signal in tqdm(enumerate(X_ecg)):
    _, _, _, _, _, _, heart_rate = ecg.ecg(signal=signal, sampling_rate=300, show=False)
    heart_rate_mean.append(np.mean(heart_rate))
    heart_rate_std.append(np.std(heart_rate))
    heart_rate_median.append(np.median(heart_rate))

In [ ]:
heart_rate_mean = np.array(heart_rate_mean)
heart_rate_std = np.array(heart_rate_std)
heart_rate_median = np.array(heart_rate_median)

### Peaks of ECG

In [ ]:
p_peaks = []
q_peaks = []
rpeaks = []
s_peaks = []
t_peaks = []

for i in range(mean_heart_full.shape[0]):
    heart = mean_heart_full[i]
    r_idx = np.argmax(heart)
    rrpeak = heart[r_idx]

    q_idx = r_idx - np.where((np.diff(heart[:r_idx][::-1]) < 0) == False)[0][0]
    q_peak = heart[q_idx]

    p_idx = np.argmax(heart[:q_idx])
    p_peak = heart[p_idx]

    s_idx = np.where((np.diff(heart[r_idx:]) < 0) == False)[0][0] + r_idx
    s_peak = heart[s_idx]

    t_idx = np.argmax(heart[s_idx:]) + s_idx
    t_peak = heart[t_idx]
    
    p_peaks.append(p_peak)
    q_peaks.append(q_peak)
    rpeaks.append(rrpeak)
    s_peaks.append(s_peak)
    t_peaks.append(t_peak)

In [ ]:
## PR, QRS, ST interval / segment
pr = []
qrs = []
st = []

for i in range(len(rpeaks)):
    pr.append(rpeaks[i] - p_peaks[i])
    qrs.append(s_peaks[i] - q_peaks[i])
    st.append(t_peaks[i] - s_peaks[i])

In [ ]:
# to numpy
rpeaks = np.array(rpeaks)
p_peaks = np.array(p_peaks)
q_peaks = np.array(q_peaks)
t_peaks = np.array(t_peaks)
s_peaks = np.array(s_peaks)
pr = np.array(pr)
qrs = np.array(qrs)
st = np.array(st)

### Wavelet transform

In [ ]:
A_list = []
D_list = []

for i in range(len(mean_heart_full)):
    A, D = pywt.dwt(mean_heart_full[i], 'db2', mode='periodic')
    A_list.append(A)
    D_list.append(D)
    
A_list = np.array(A_list)
D_list = np.array(D_list)

## Gather all features

In [ ]:
X_features = np.column_stack((rpeakmean, rpeakmstd, rpeakmedian, rpeakmax, rpeakmin, mean_h, std_h, median_h,
                              max_mean_h, min_mean_h, max_std_h, min_std_h, heart_rate_mean, heart_rate_std,
                              heart_rate_median, rpeaks, p_peaks, q_peaks, s_peaks, t_peaks, pr, qrs, st,
                              hrv_time_domain, A_list, D_list))

## Split data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.1, random_state=rnd, stratify=y['y'])

## Model

In [ ]:
xgbc = XGBClassifier(seed = rnd, n_jobs=-1, eval_metric='mlogloss', objective='multi:softmax', num_class=4)
xgb_grid = {'max_depth': [6], 
            'n_estimators': [500],
            'reg_lambda': [10],
            'eta': [0.03],
            'reg_alpha': [2],
            'min_child_weight': [4]
           }

In [ ]:
X_train_std = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_val_std = (X_val - X_val.mean(axis=0)) / X_val.std(axis=0)
X_features_std = (X_features - X_features.mean(axis=0)) / X_features.std(axis=0)

In [ ]:
gcv = GridSearchCV(xgbc, xgb_grid, cv=3, scoring='f1_micro', n_jobs=1,
                   verbose=10)

np.random.seed(rnd)
# gcv.fit(X_train_std, y_train) 
gcv.fit(X_features_std, y)

In [ ]:
gcv.best_score_, gcv.best_params_

In [ ]:
y_pred_train = gcv.predict(X_train_std)
y_pred_val = gcv.predict(X_val_std)

In [ ]:
# train
f1_score(y_train, y_pred_train, average='micro')
# val
f1_score(y_val, y_pred_val, average='micro')

## Test

### Same feature extraction as above

In [ ]:
test_clean = [item[~np.isnan(item)] for item in test.values]
test_ecg = list(map(clean_raw_ecg, test_clean))

In [ ]:
# extract rpeaks and heartbeat stats
r_peaks_test = []
mean_hear_t = []
std_hear_t = []
median_hear_t = []
for idx, signal in tqdm(enumerate(test_ecg)):
    _, _, peak, _, templates, _, _ = ecg.ecg(signal=signal, sampling_rate=300, show=False)
    r_peaks_test.append(peak)
    mean_hear_t.append(np.mean(templates, axis=0))
    std_hear_t.append(np.std(templates, axis=0))
    median_hear_t.append(np.median(templates, axis=0))

In [ ]:
mean_heart_test = np.vstack(mean_hear_t)
std_heart_test = np.vstack(std_hear_t)
median_heart_test = np.vstack(median_hear_t)

In [ ]:
peak_values_test = []
for idx, val in enumerate(test_ecg):
    peakst = np.array(r_peaks_test[idx])
    peak_values_test.append((np.array(test_ecg[idx][peakst])))        

In [ ]:
# extract stats about the r-peaks
rpeak_mean = np.array([item.mean() for item in peak_values_test])
rpeak_median = np.array([np.median(item) for item in peak_values_test])
rpeakm_std = np.array([item.std() for item in peak_values_test])
rpeak_max = np.array([item.max() for item in peak_values_test])
rpeak_min = np.array([item.min() for item in peak_values_test])

In [ ]:
# extract features related to heartbeat
meanh = np.median(mean_heart_test, axis=1)
stdh = np.median(std_heart_test, axis=1)
medianh = np.median(median_heart_test, axis=1)
max_meanh = np.max(mean_heart_test, axis = 1)
min_meanh = np.min(mean_heart_test, axis = 1)
max_stdh = np.max(std_heart_test, axis = 1)
min_stdh = np.min(std_heart_test, axis = 1)

In [ ]:
hrv_time_domain_test = pd.DataFrame(nk.hrv_time(r_peaks_test[0], sampling_rate=300))
idx_list = []
for idx, peak in enumerate(r_peaks_test[1:]):
    hrv_time_domain_test = pd.concat([hrv_time_domain_test, nk.hrv_time(peak, sampling_rate=300)])

hrv_time_domain_test = hrv_time_domain_test.dropna(axis=1)
hrv_time_domain_test = np.array(hrv_time_domain_test)

In [ ]:
## heartrate
heart_rate_mean_test = []
heart_rate_std_test = []
heart_rate_median_test = []
for idx, signal in tqdm(enumerate(test_ecg)):
    _, _, _, _, _, _, heart_rate = ecg.ecg(signal=signal, sampling_rate=300, show=False)
    heart_rate_mean_test.append(np.mean(heart_rate))
    heart_rate_std_test.append(np.std(heart_rate))
    heart_rate_median_test.append(np.median(heart_rate))

In [ ]:
heart_rate_mean_test = np.array(heart_rate_mean_test)
heart_rate_std_test = np.array(heart_rate_std_test)
heart_rate_median_test = np.array(heart_rate_median_test)

In [ ]:
p_peakstest = []
q_peakstest = []
rpeakstest = []
s_peakstest = []
t_peakstest = []

for i in range(mean_heart_test.shape[0]):
    try:
        heart = mean_heart_test[i]
        r_idx = np.argmax(heart)
        rrpeak = heart[r_idx] # max = r-peak

        q_idx = r_idx - np.where((np.diff(heart[:r_idx][::-1]) < 0) == False)[0][0]
        q_peak = heart[q_idx]

        p_idx = np.argmax(heart[:q_idx])
        p_peak = heart[p_idx]

        s_idx = np.where((np.diff(heart[r_idx:]) < 0) == False)[0][0] + r_idx
        s_peak = heart[s_idx]

        t_idx = np.argmax(heart[s_idx:]) + s_idx
        t_peak = heart[t_idx]

        p_peakstest.append(p_peak)
        q_peakstest.append(q_peak)
        rpeakstest.append(rrpeak)
        s_peakstest.append(s_peak)
        t_peakstest.append(t_peak)
    except:
        print(i)
        heart = mean_heart_test[i]
        r_idx = np.argmax(heart)
        rrpeak = heart[r_idx]
        
        q_idx = 0
        q_peak = heart[q_idx]
        
        p_idx = 0
        p_peak = heart[p_idx]
        
        s_idx = np.where((np.diff(heart[r_idx:]) < 0) == False)[0][0] + r_idx
        s_peak = heart[s_idx]
        
        t_idx = np.argmax(heart[s_idx:]) + s_idx
        t_peak = heart[t_idx]

        p_peakstest.append(p_peak)
        q_peakstest.append(q_peak)
        rpeakstest.append(rrpeak)
        s_peakstest.append(s_peak)
        t_peakstest.append(t_peak)

In [ ]:
## PR, QRS, ST interval / segment

prtest = []
qrstest = []
sttest = []

for i in range(len(rpeakstest)):
    prtest.append(rpeakstest[i] - p_peakstest[i])
    qrstest.append(s_peakstest[i] - q_peakstest[i])
    sttest.append(t_peakstest[i] - s_peakstest[i])

In [ ]:
# to numpy
rpeakstest = np.array(rpeakstest)
p_peakstest = np.array(p_peakstest)
q_peakstest = np.array(q_peakstest)
t_peakstest = np.array(t_peakstest)
s_peakstest = np.array(s_peakstest)
prtest = np.array(prtest)
qrstest = np.array(qrstest)
sttest = np.array(sttest)

In [ ]:
A_list_test = []
D_list_test = []
for i in range(len(mean_heart_test)):
    A, D = pywt.dwt(mean_heart_test[i], 'db2', mode='periodic')
    A_list_test.append(A)
    D_list_test.append(D)
    
A_list_test = np.array(A_list_test)
D_list_test = np.array(D_list_test)

### Collect features

In [ ]:
test_features = np.column_stack((rpeak_mean, rpeakm_std, rpeak_median, rpeak_max, rpeak_min, meanh, stdh, medianh,
                                 max_meanh, min_meanh, max_stdh, min_stdh, heart_rate_mean_test, heart_rate_std_test,
                                 heart_rate_median_test, rpeakstest, p_peakstest, q_peakstest, s_peakstest, t_peakstest,
                                 prtest, qrstest, sttest, hrv_time_domain_test, A_list_test, D_list_test))

In [ ]:
test_features_std = (test_features - test_features.mean(axis=0)) / test_features.std(axis=0)
y_test_pred = gcv.predict(test_features_std)

## Output

In [ ]:
output = np.column_stack((np.array(test_ids), y_test_pred))

df = pd.DataFrame(output, columns=['id', 'y'])
df.to_csv('submission.csv', index=False, header=True, sep=',')